#ENTBERT

BERT model- last layer takes the embedding of BERT from the index of analysed entity and concatenate it with the embedding of CLS token

datasets available https://zenodo.org/records/5211931
extended datasets available on a request

## Read data

In [ ]:
import pandas as pd


df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sen/texts_pl.csv')




In [ ]:
df_train = df.sample(frac=0.75, random_state=0)
df_test = df.drop(df_train.index)
df_train = df_train.reset_index()
df_test = df_test.reset_index()

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download pl_core_news_sm

2023-06-09 21:28:11.030792: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 21:28:12.065594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 21:28:14.320230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-09 21:28:14.320817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
# from spacy.lang.en import English
import spacy
def find_entities(df, spacy_model="pl_core_news_sm"):
  nlp = spacy.load(spacy_model)

  all= pd.DataFrame(columns=["sentence","tokens","pos", "entities", "polarity","id","target"])

  for i, (sentence, annotations) in enumerate(zip(df["sentence"],df["polarity"])):
      doc=nlp(sentence)
      token_list=[]
      pos_list=[]
      entities_list=[]
      entities_article_list=[]



      #czesci mowy
      for token in doc:
          token_list.append(str(token.text))
          pos_list.append(str(token.pos_))


      #named entity headline
      for ent in doc.ents:
          entities_list.append(ent.text)
          # entities_labels_list.append(ent.label_)


      #typ encji
      mention=df["entity"][i]
      mention_type=""
      doc_mention=nlp(mention)
      for ent in doc_mention.ents:
        mention_type=ent.label_
      if mention_type=="":
        mention_type= df["entity"][i]


      # df["tokens"][i]=token_list
      tar=df.mention[i]
      id=df.id[i]
      tmp=pd.DataFrame({"sentence":sentence,"tokens":", ".join(token_list), "pos":", ".join(pos_list),
                        "entities": ", ".join(entities_list),"polarity":annotations,"id":id, "target": tar, "target_type":mention_type, "from": df["from"][i], "to":df.to[i]},index=["id"])

      all=pd.concat([all,tmp], ignore_index=True)

  return all

In [ ]:
df_train=find_entities(df_train)


In [ ]:
df_test=find_entities(df_test)
df_dev=find_entities(df_dev)


In [ ]:
df_train

,sentence,tokens,pos,entities,polarity,id,target,target_type,from,to
0,Winner wrote that she had a 30-minute private ...,"Winner, wrote, that, she, had, a, 30, -, minut...","PROPN, VERB, SCONJ, PRON, VERB, DET, NUM, PUNC...","Winner, 30-minute, Republican",0,allsides_1000_401_25_Reality Leigh Winner_0_6,Winner,PERSON,0.0,6.0
1,She also recently referred to President Trump ...,"She, also, recently, referred, to, President, ...","PRON, ADV, ADV, VERB, ADP, PROPN, PROPN, ADP, ...","Trump, the Dakota Access Pipeline",-1,allsides_1000_401_3_Reality Leigh Winner_0_3,She,She,0.0,3.0
2,She also recently referred to President Trump ...,"She, also, recently, referred, to, President, ...","PRON, ADV, ADV, VERB, ADP, PROPN, PROPN, ADP, ...","Trump, the Dakota Access Pipeline",-1,allsides_1000_401_3_’ Trump_30_45,President Trump,PERSON,30.0,45.0
3,Hillary Clinton blamed the Democratic National...,"Hillary, Clinton, blamed, the, Democratic, Nat...","PROPN, PROPN, VERB, DET, PROPN, PROPN, PROPN, ...","Hillary Clinton, the Democratic National Commi...",-1,allsides_1018_408_1_Hillary Clinton_2_17,Hillary Clinton,PERSON,0.0,15.0
4,Hillary Clinton blamed the Democratic National...,"Hillary, Clinton, blamed, the, Democratic, Nat...","PROPN, PROPN, VERB, DET, PROPN, PROPN, PROPN, ...","Hillary Clinton, the Democratic National Commi...",-1,allsides_1018_408_1_Facebook_60_68,Facebook,Facebook,58.0,66.0
...,...,...,...,...,...,...,...,...,...,...
8734,"In announcing Acosta's suspension, Sanders sai...","In, announcing, Acosta, 's, suspension, ,, San...","ADP, VERB, PROPN, PART, NOUN, PUNCT, PROPN, VE...","Acosta, Sanders, CNN",-1,polusa_v1_59576418_-1_41_CNN correspondent Jim...,Acosta,PERSON,14.0,20.0
8735,Cassidy has sought to defend the provision.,"Cassidy, has, sought, to, defend, the, provisi...","NOUN, AUX, VERB, PART, VERB, DET, NOUN, PUNCT",Cassidy,0,allsides_569_226_44_Graham-Cassidy_0_7,Cassidy,ORG,0.0,7.0
8736,Trump returned to the United States on Tuesday...,"Trump, returned, to, the, United, States, on, ...","PROPN, VERB, ADP, DET, PROPN, PROPN, ADP, PROP...","Trump, the United States, Tuesday, night",0,polusa_v1_4359001_-1_13_Donald Trump_1_6,Trump,ORG,0.0,5.0
8737,A Wall Street Journal/NBC poll taken after the...,"A, Wall, Street, Journal, /, NBC, poll, taken,...","DET, PROPN, PROPN, PROPN, SYM, PROPN, NOUN, VE...","Wall Street Journal/NBC, the “Access Hollywood...",1,allsides_1732_705_20_Hillary Clinton_104_111,Clinton,PERSON,104.0,111.0


In [ ]:
df_train["label"]=df_train["polarity"]
df_test["label"]=df_test["polarity"]
df_dev["label"]=df_dev["polarity"]

In [ ]:
df_train=df_train.drop("polarity", axis=1)
df_test=df_test.drop("polarity", axis=1)
df_dev=df_dev.drop("polarity", axis=1)

In [ ]:
#przygotownaie etykiet do klasyfikacji
labels_to_ids = {k: v for v, k in enumerate(set(df_train["label"]))}
ids_to_labels = {v: k for v, k in enumerate(set(df_train["label"]))}
labels_to_ids

{0: 0, 1: 1, -1: 2}

## Dataset

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizerFast, BertConfig, BertModel

#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

model_type="roberta"#robert
if model_type=="bert":
  bert_name= 'bert-base-cased'
  tokenizer = BertTokenizerFast.from_pretrained(bert_name)
if model_type=="polbert":
  bert_name="dkleczek/bert-base-polish-uncased-v1"
  tokenizer = BertTokenizerFast.from_pretrained(bert_name)


if model_type=="roberta":
  bert_name="roberta-base"
  tokenizer = AutoTokenizer.from_pretrained(bert_name)
  model = AutoModel.from_pretrained(bert_name)

elif model_type=="herbert":
  !pip install sacremoses
  bert_name="allegro/herbert-klej-cased-v1"
  tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
  model = AutoModel.from_pretrained("allegro/herbert-klej-cased-v1")

elif model_type=="roberta_clarin":
  roberta_name= 'clarin-pl/roberta-polish-kgr10'
  tokenizer = AutoTokenizer.from_pretrained(roberta_name)
  model = AutoModel.from_pretrained(roberta_name)

elif model_type=="roberta_polish_dadas":
  roberta_name= 'sdadas/polish-roberta-base-v2'
  tokenizer = AutoTokenizer.from_pretrained(roberta_name)
  model = AutoModel.from_pretrained(roberta_name)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

import numpy as np
import torch



class MyDataset(Dataset):

  def __init__(self,df, tokenizer, max_len, typed, dataset):

    if dataset=="sen":
      self.sentence=df["headline"]
    else:
      self.sentence=df["sentence"]

    if dataset=="sen":
      ee= [sen.split(ent) for sen, ent in zip(self.sentence, df["entity_clean"])]
      self.entity=df["entity_clean"]

      self.ind_ent=[len(e[0])+len(self.entity) if len(e)==2 else len(self.entity) for e in ee]
    else:

      self.ind_ent=df["to"]
      self.entity=df["target"]
    ee= [sen.split(ent) for sen, ent in zip(self.sentence, self.entity)]

    if typed:
      self.entype=df["target_type"]

      self.sentence=[e[0] +self.entype[i] +e[1] for i, e in enumerate(ee)]

    self.encoding = tokenizer(self.sentence,  #przenies do wyzej
                             is_split_into_words=False,

                             padding='max_length',
                             truncation=True,
                             max_length=max_len)#.set_format("torch")

    self.max_len=max_len

    self.label=[labels_to_ids[lab] for lab in df["label"].fillna("Neutral")]




  def __len__(self):
    return len(self.label)

  def __getitem__(self,idx):

    labels = self.label[idx]

    items = {key: torch.as_tensor(val[idx]) for key, val in self.encoding.items()}

    items["labels"]=torch.as_tensor(labels)

    items["entity_ind"]=self.ind_ent[idx]
    if self.ind_ent[idx]>=self.max_len:
      items["entity_ind"]=self.max_len-1


    items["entitiy"]=self.entity[idx]

    return items

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:

print("TRAIN Dataset: {}".format(df_train.shape))
print("TEST Dataset: {}".format(df_test.shape))
print("DEV Dataset: {}".format(df_dev.shape))


training_set = MyDataset(df_train, tokenizer, MAX_LEN, True, "mt")
test_set = MyDataset(df_test, tokenizer, MAX_LEN, True, "mt")
dev_set = MyDataset(df_dev, tokenizer, MAX_LEN, True, "mt")



# myDs=MyDataset(bias_lexical, tokenizer)
train_loader=DataLoader(training_set,batch_size=TRAIN_BATCH_SIZE,shuffle=True)
test_loader=DataLoader(test_set,batch_size=VALID_BATCH_SIZE, shuffle=False)
dev_loader=DataLoader(dev_set,batch_size=VALID_BATCH_SIZE, shuffle=False)

TRAIN Dataset: (8739, 10)
TEST Dataset: (1034, 10)
DEV Dataset: (442, 10)


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


## Model

In [ ]:

import torch.nn as nn
import torch
from torch.nn import CrossEntropyLoss
from transformers.models.bert import BertPreTrainedModel

class ENTBert(nn.Module):
    def __init__(self, bert_name, num_labels):
        super().__init__()

        self.bert=AutoModel.from_pretrained(bert_name, num_labels=num_labels, add_pooling_layer=True)

        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(int(768*2), 3)
        self.pool=nn.MaxPool1d(2)

        nn.init.xavier_normal_(self.classifier.weight)
      )

    def forward(self,
        input_ids=None,
                token_type_ids=None,attention_mask=None,idx=1 ,
                labels=None):

        outputs = self.bert(
            input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids
        )

        a=torch.range(0,len(idx)-1,dtype=torch.long)

        cls_output = outputs[0][:, 0, :]

        idx_output = outputs[0][a, idx, :]


        pooled_output=torch.cat((cls_output,idx_output), dim=1)


        if len(pooled_output.shape)==1:
            pooled_output=pooled_output.unsqueeze(0)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        #
        return logits



In [ ]:
class_weights=[]
for i in labels_to_ids.keys():
    class_weights.append((len(df_train)-sum(df_train["label"]==i))/(sum(df_train["label"]==i)))
class_weights

[1.8860634081902246, 2.6488517745302715, 1.6354041013268998]

In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertModel
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch
#trenowanie



class_weights=torch.as_tensor(class_weights).to(device)

loss_fct = CrossEntropyLoss(weight=class_weights)


def train(epoch, model, test=True, test_loader=test_loader):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    print("Start")
    for idx, batch in enumerate(train_loader):
        optimizer.zero_grad()

        ids1 = batch['input_ids'].to(device, dtype = torch.long)
        mask1 = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)
        ent_idx=batch["entity_ind"].to(device, dtype = torch.long)





        output = model(input_ids=ids1, attention_mask=mask1, idx=ent_idx, labels=None)
        loss=loss_fct(output, labels)
#
        tr_loss += loss.item()
        tr_logits=output

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")


        predictions = torch.argmax(output,  dim=1) # shape (batch_size * seq_len,) labels?

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = sum(labels==predictions)#accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

    tr_labels=[lab.item() for lab in tr_labels ]
    tr_preds=[lab.item() for lab in tr_preds ]

    print(classification_report(tr_labels, tr_preds))


    print("Test eval")
    model.eval()
    te_preds, te_labels = [], []
    loss=0
    if test:
      for batch in test_loader:
        # print(batch)
        # batch = {k: v.to(device) for k, v in batch.items()}
        ids1 = batch['input_ids'].to(device, dtype = torch.long)
        mask1 = batch['attention_mask'].to(device, dtype = torch.long)
        ent_idx=batch["entity_ind"].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)




        with torch.no_grad():
            # print(ids)
            output = model(input_ids=ids1, attention_mask=mask1, idx=ent_idx, labels=None)
            loss+=loss_fct(output, labels)
            preds=torch.argmax(output,  dim=1)
            te_labels.extend(labels)
            te_preds.extend(preds)
      te_labels=[lab.item() for lab in te_labels ]
      te_preds=[lab.item() for lab in te_preds ]
      loss_test=loss/len(test_loader)
      print(classification_report(te_labels, te_preds))
      print(accuracy_score(te_labels, te_preds))
      print(f1_score(te_labels, te_preds, average="macro"))


      print(loss_test)


In [ ]:
model= ENTBert(bert_name, num_labels=len(labels_to_ids)).to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch, model, True, dev_loader)

Training epoch: 1
Start


<ipython-input-20-c5d0f89493ed>:39: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  a=torch.range(0,len(idx)-1,dtype=torch.long)


Training loss per 100 training steps: 1.2850472927093506
Training loss per 100 training steps: 1.0582505293411784
Training loss per 100 training steps: 0.9666486178612828
Training loss per 100 training steps: 0.9127343051952381
Training loss per 100 training steps: 0.8729316968647322
Training loss per 100 training steps: 0.8462213377157847
Training loss per 100 training steps: 0.8236086129299218
Training loss per 100 training steps: 0.7968435784676956
Training loss per 100 training steps: 0.7728718306018172
Training loss per 100 training steps: 0.7596688116926464
Training loss per 100 training steps: 0.7429234258063904
Training loss epoch: 0.7296092298498748
Training accuracy epoch: 5.413540840148926
              precision    recall  f1-score   support

           0       0.63      0.57      0.60      3028
           1       0.65      0.75      0.70      2395
           2       0.74      0.72      0.73      3316

    accuracy                           0.68      8739
   macro avg      

<ipython-input-20-c5d0f89493ed>:39: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  a=torch.range(0,len(idx)-1,dtype=torch.long)


              precision    recall  f1-score   support

           0       0.68      0.94      0.79       217
           1       0.82      0.56      0.67        89
           2       0.89      0.53      0.66       136

    accuracy                           0.74       442
   macro avg       0.80      0.68      0.71       442
weighted avg       0.77      0.74      0.73       442

0.7375565610859729
0.7064764717475768
tensor(0.6158, device='cuda:0')
Training epoch: 2
Start
Training loss per 100 training steps: 0.5298881530761719


...

              precision    recall  f1-score   support

           0       0.76      0.81      0.78       217
           1       0.82      0.63      0.71        89
           2       0.77      0.79      0.78       136

    accuracy                           0.77       442
   macro avg       0.78      0.74      0.76       442
weighted avg       0.77      0.77      0.77       442

0.7692307692307693
0.7584604706341237
tensor(1.0986, device='cuda:0')
Training epoch: 10
Start
Training loss per 100 training steps: 0.001886256504803896


<ipython-input-20-c5d0f89493ed>:39: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  a=torch.range(0,len(idx)-1,dtype=torch.long)


Training loss per 100 training steps: 0.07352309034712841
Training loss per 100 training steps: 0.054663533561840415
Training loss per 100 training steps: 0.05485996582089541
Training loss per 100 training steps: 0.05403076875868498
Training loss per 100 training steps: 0.05037953577169004
Training loss per 100 training steps: 0.04898328355986644
Training loss per 100 training steps: 0.04969814743199562
Training loss per 100 training steps: 0.05152777844539494
Training loss per 100 training steps: 0.05330472270811702
Training loss per 100 training steps: 0.053193973667055125
Training loss epoch: 0.05299295635313808
Training accuracy epoch: 7.85361385345459
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3028
           1       0.98      0.99      0.99      2395
           2       0.99      0.98      0.98      3316

    accuracy                           0.98      8739
   macro avg       0.98      0.98      0.98      8739
weighted 

<ipython-input-20-c5d0f89493ed>:39: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  a=torch.range(0,len(idx)-1,dtype=torch.long)


              precision    recall  f1-score   support

           0       0.82      0.69      0.75       217
           1       0.72      0.71      0.72        89
           2       0.68      0.87      0.76       136

    accuracy                           0.75       442
   macro avg       0.74      0.76      0.74       442
weighted avg       0.76      0.75      0.75       442

0.748868778280543
0.7438476118215515
tensor(1.1233, device='cuda:0')
